# YOLOv5 experiment

In [11]:
import pandas as pd
import numpy as np
import glob
import os

## Import youtube-bb annotations of the validation test

In [190]:
ytbb = pd.read_csv("./youtube_boundingboxes_detection_validation.csv", header = None, 
                   names = ["youtube_id", "timestamp_ms", "class_id", "class_name", "object_id", "object_presence",
                            "xmin", "xmax", "ymin", "ymax"])

In [191]:
ytbb = ytbb.sort_values(['youtube_id', 'class_id', 'object_id', 'timestamp_ms'])

In [192]:
ytbb.head()

,youtube_id,timestamp_ms,class_id,class_name,object_id,object_presence,xmin,xmax,ymin,ymax
614707,--6SUm2LGfQ,0,1,bird,0,present,0.403,0.539,0.220000,0.553333
614708,--6SUm2LGfQ,1000,1,bird,0,present,0.408,0.548,0.206667,0.563333
614709,--6SUm2LGfQ,2000,1,bird,0,present,0.413,0.531,0.156667,0.525000
614710,--6SUm2LGfQ,3000,1,bird,0,present,0.419,0.519,0.150000,0.485000
614711,--6SUm2LGfQ,4000,1,bird,0,present,0.375,0.529,0.133333,0.568333


## Import our results and merge the csv files

In [207]:
extension = 'csv'
all_filenames = [i for i in glob.glob('inference\\output\\*.{}'.format(extension))]
pred = pd.concat([pd.read_csv(f, index_col = 0) for f in all_filenames])

In [264]:
pred.head()

,youtube_id,class_id,object_id,image_nb,xmin,xmax,ymin,ymax,found_id,found_prec
0,--6SUm2LGfQ,1,0,2,212.0,86.0,261.0,203.0,0,0.436980
1,--6SUm2LGfQ,1,0,3,30.0,263.0,81.0,343.0,15,0.647754
2,--6SUm2LGfQ,1,0,3,0.0,117.0,68.0,245.0,3,0.630265
3,--6SUm2LGfQ,1,0,11,180.0,81.0,280.0,356.0,0,0.592035
4,--6SUm2LGfQ,1,0,14,178.0,38.0,289.0,354.0,0,0.464583


## Go through all the annotation frames and see if they match with our predictions

In [222]:
ytbb2yolo = [0, 14, 1, 8, 5, 21, 19, 15, 23, 58, 17, 3, 43, 4, 36, 6, 7, 22, 61, 16, 20, 25, -1, 2]
yt_id_all = np.unique(pred.youtube_id)
n,c = 0,0
rate_list = []
nb_frame = 0
for k,yt_id in enumerate(yt_id_all):
  if k%100 == 0:
    print("video",k)
  n_vid = 0
  c_vid = 0
  pred_r = pred[pred.youtube_id == yt_id]
  ytbb_r = ytbb[ytbb.youtube_id == yt_id]
  for c_id in np.unique(ytbb_r.class_id):
    for o_id in np.unique(ytbb_r[ytbb_r.class_id == c_id].object_id):
      pred_rr = pred_r[(pred_r.class_id == c_id) & (pred_r.object_id == o_id) ]
      ytbb_rr = ytbb_r[(ytbb_r.class_id == c_id) & (ytbb_r.object_id == o_id) ]
      for i in range(len(ytbb_rr)):
        ytbb_row = ytbb_rr.iloc[i]
        nb_frame+=1
        if ytbb_row.object_presence == "present":
          n+=1
          n_vid+=1
          found = False
          pred_rows = pred_rr[(pred_rr.class_id == ytbb_row.class_id) & \
                        (pred_rr.object_id == ytbb_row.object_id) & \
                        (pred_rr.image_nb == (i+1))]
         
          for j in range(len(pred_rows)):
            pred_row = pred_rows.iloc[j]
            if pred_row.found_id == ytbb2yolo[int(ytbb_row.class_id)]:
              found = True
          c+=found
          c_vid+=found
  if (n_vid != 0):
    rate_list.append(c_vid/n_vid)
  else:
      rate_list.append(-1)
rate = c/n

video 0
video 100
video 200
video 300
video 400
video 500
video 600
video 700
video 800
video 900
video 1000
video 1100
video 1200
video 1300
video 1400
video 1500
video 1600


## Results

In [223]:
rate

0.5603700342811051

In [239]:
nb_frame

49717